# Calculate Embeddings

Ajuda a calcular embeddings dos dados adquiridos dos comentários das redes sociais de políticos. (Planilha de Gabrielly)

Entrada: `LulaTotal Validado.xlsx` e `Bolsonaro Validado.xlsx`

Saída: `Embeddings_(NOME_DO_MODELO)_(REDE_SOCIAL).xlsx`

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer

BASE_PATH = 'dados/'
DIRECTORY = BASE_PATH + 'preprocessed/embeddings/'
SOCIAL_NETWORK = 'tiktok'

In [ ]:
column_types = {'ID' : str}

# 1. Leitura de Arquivos e inicialização de Variáveis

Iniciação das variáveis que iremos utilizar e das colunas na qual iremos trabalhar


In [ ]:
# 1. Pegar o modelo para testar
TYPE_MODEL = 'BAAI/bge-m3'
model = SentenceTransformer(TYPE_MODEL)

# 2. Pegar as sentenças (nesse caso, no Post-filtrado)
file_pathL = BASE_PATH + 'input/LulaTotal Validado.xlsx'
file_pathB = BASE_PATH + 'input/Bolsonaro Validado.xlsx'
file_path_features = 'Embeddings_bge-m3_'+ (SOCIAL_NETWORK) +'.xlsx'
#é bom modificar o nome manualmente, pois pode dar erro na questão do save

column_text = "Texto"
column_id = "ID"
column_author = "Perfil"
column_likes = "Curtidas"
column_inicial_date = "DataColeta"
column_final_date = "DataPost"
column_cand = "Candidato"

In [ ]:
# converte para datetime 
def adjust_dates(df_aux):
    df_aux['DataColeta'] = pd.to_datetime(df_aux['DataColeta'])
    df_aux['DataPost'] = pd.to_datetime(df_aux['DataPost'])

    try:
        # converte esta coluna para conter uma string com o dia da semana
        df_aux['DiaDaSemana'] = pd.to_datetime(df_aux['DiaDaSemana'])
        df_aux['DiaDaSemana'] = df_aux['DiaDaSemana'].dt.strftime('%A')
    except:
        print("Coluna DiaDaSemana parece já estar guardado como date. Não precisa transformar!")

In [ ]:
# 3. Ler os Arquivos tanto de Lula quanto de bolsonaro
df_lula = pd.read_excel(file_pathL, dtype=column_types)
df_bolsonaro = pd.read_excel(file_pathB, dtype=column_types)

df_lula.shape, df_bolsonaro.shape


União das planilha e limpeza de variáveis que estão em branco, ou seja, não possuem utilidade.

In [ ]:
df_total = pd.concat([df_lula, df_bolsonaro], axis=0)
df_total.shape

In [ ]:
len(df_total['ID'].unique())


In [ ]:
df_total = df_total.reset_index(drop=True)
df_total.head(5)

In [ ]:
df_total.dropna(inplace=True)

Observar o que foi eliminado e conferir se está tudo certo:

In [ ]:
df_total.shape

In [ ]:
df_total[df_total.isna().sum(axis=1) > 0].groupby('Perfil').count()

In [ ]:
df_total.loc[df_total.isna().sum(axis=1) > 0].shape
df_total.shape

# 2. Configuração da planilha pré-processada
## 2.1 Limpeza de colunas 

Drop de colunas que não iremos utilizar como por exemplo `Duração` ou até `Dispositivo Retórico` etc.

In [ ]:
df_total.drop(columns=["Unnamed: 0","Plays","Comentarios","Compart.","Duracao", "LinkFoto", "LinkVideo", "LinkPost", 
                       "Retórica Aristotélica", "Dispositivo Retórico", "Tipo de conteúdo",
                  "Abordagem", "Tonalidade", "Main character", "Texto / Hashtag"], inplace=True)
df_total.tail(5)

In [ ]:
df_total.loc[df_total.isna().sum(axis=1) > 0].shape
df_total.dropna(inplace=True)
df_total.shape

Conversão de datas para calcular a coluna de `Dias Decorridos`:

In [ ]:
# converte para datetime
adjust_dates(df_total)
df_total.tail(5)

In [ ]:
df_total.groupby(['DataColeta']).count()

In [ ]:
df_total['DiasDecorridos'] = (df_total['DataColeta'] - df_total['DataPost']).dt.days
df_total

df_total['Candidato'] = df_total['Perfil'].apply(lambda x: 'Lula' if x == 'lulaoficial' else 'Bolsonaro')

## 2.2 Flag para análise de Hashtags

Função para verificar se o texto contém apenas hashtags

Caso o comentário só possua hashtags ele irá retornar  `true`, caso contrário a função retorna `false`

In [ ]:
import re

def contains_only_hashtags(text):
    hashtags = re.findall(r'#\S+', text)
    return len(hashtags) == len(text.split())

In [ ]:
ids = df_total[column_id].tolist()
authors = df_total[column_author].tolist()
sentences = df_total[column_text].tolist()
likes = df_total[column_likes].tolist()
days = df_total['DiasDecorridos'].tolist()
candidatos = df_total['Candidato'].tolist()

Encode dos embeddings utilizando a library do "sentence-transformers" e também a concatenação de dados previamente informados da planilha passada como `ID`,`Candidato` e `Curtidas`

In [ ]:
# 4. Calcular os embeddings das sentenças
embeddings = model.encode(sentences)
df_embeddings = pd.DataFrame(embeddings) 
df_embeddings.columns = [f'x{i+1}' for i in range(df_embeddings.shape[1])]



In [ ]:
df_embeddings.tail(5)

A planilha ficará com as colunas importantes originais (__ID__,__Candidato__ e __Curtidas__), e também estará com uma flag que mostra se há hashtags(__Only Hashtags__), e por fim, features calculadas pelo embeddinG   __x1__,__x2__,__x3__ ... __xN__ , onde N seria o tamanho de dimensões que aquele modelo possui

# 3. Concatenar e Salvar o Arquivo

In [ ]:
df_final = pd.DataFrame({
    column_id: ids,
    'Candidato': candidatos,
    column_likes: likes,
    'Dias Decorridos': days
    
})

df_final['Only Hashtags'] = df_total[column_text].apply(contains_only_hashtags)
df_final['Only Hashtags'] = df_final['Only Hashtags'].fillna(False)  # Para lidar com tabulações que o pandas não reconhece

# Concatena com df_embeddings
df_final = pd.concat([df_final, df_embeddings], axis=1)


In [ ]:
df_final.tail(5)


## 3.1 Salvar o arquivo

In [ ]:

df_final.to_excel((DIRECTORY + file_path_features), index=False)

print("Arquivo salvo:", file_path_features)